# **The Storm Of the Century: Temperature Advection**
***

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
import metpy.calc as mpcalc
from scipy import ndimage  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import metpy
from metpy.plots import ctables
from matplotlib.colors import LinearSegmentedColormap

## **Date Input**

In [ ]:
startYear = 1993
startMonth = 3
startDay = 7
startHour = 0
startMinute = 0
startDateTime = dt(startYear,startMonth,startDay, startHour, startMinute)

endYear = 1993
endMonth = 3
endDay = 15
endHour = 0
endMinute = 0
endDateTime = dt(endYear,endMonth,endDay, endHour, endMinute)

In [ ]:
dateList = pd.date_range(startDateTime, endDateTime,freq="6H")
dateList

## **CFSR**

In [ ]:
dsZ = xr.open_dataset (f'/cfsr/data/{startYear}/g.{startYear}.0p5.anl.nc')
dsT = xr.open_dataset (f'/cfsr/data/{startYear}/t.{startYear}.0p5.anl.nc')
dsU = xr.open_dataset (f'/cfsr/data/{startYear}/u.{startYear}.0p5.anl.nc')
dsV = xr.open_dataset (f'/cfsr/data/{startYear}/v.{startYear}.0p5.anl.nc')
dsW = xr.open_dataset (f'/cfsr/data/{startYear}/w.{startYear}.0p5.anl.nc')
dsQ = xr.open_dataset (f'/cfsr/data/{startYear}/q.{startYear}.0p5.anl.nc')
dsSLP = xr.open_dataset (f'/cfsr/data/{startYear}/pmsl.{startYear}.0p5.anl.nc')

## **Area and Domain**

In [ ]:
lonW = -99
lonE = -59
latS = 20
latN = 50
cLat, cLon = (latS + latN)/2, (lonW + lonE)/2
latRange = np.arange(latS,latN+.5,.5) # expand the data range a bit beyond the plot range
lonRange = np.arange(lonW,lonE+.5,.5) # Need to match longitude values to those of the coordinate variable
constrainLat, constrainLon = (0.6, 7)
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

## **Variables**

In [ ]:
plevel=850
Z = dsZ['g'].sel(time=dateList,lev=plevel,lat=latRange,lon=lonRange)
U = dsU['u'].sel(time=dateList,lev=plevel,lat=latRange,lon=lonRange)
V = dsV['v'].sel(time=dateList,lev=plevel,lat=latRange,lon=lonRange)
T = dsT['t'].sel(time=dateList,lev=plevel,lat=latRange,lon=lonRange)
Q = dsQ['q'].sel(time=dateList,lev=plevel,lat=latRange,lon=lonRange)
SLP = dsSLP['pmsl'].sel(time=dateList,lat=latRange,lon=lonRange)

In [ ]:
crsCFSR = {'grid_mapping_name': 'latitude_longitude', 'earth_radius': 6371229.0}
Z = Z.metpy.assign_crs(crsCFSR)
U = U.metpy.assign_crs(crsCFSR)
V = V.metpy.assign_crs(crsCFSR)
T = T.metpy.assign_crs(crsCFSR)
Q = Q.metpy.assign_crs(crsCFSR)
SLP = SLP.metpy.assign_crs(crsCFSR)

In [ ]:
lats = SLP.lat
lons = SLP.lon

## **MetPy Calculations**

In [ ]:
SLP=SLP.metpy.convert_units('hPa')
Ums = U.metpy.convert_units('m/s')
Vms = V.metpy.convert_units('m/s')
T=T.metpy.convert_units('degF')

In [ ]:
# Windspeed 
#WSPD = mpcalc.wind_speed(U,V)
######
# Divergence
#div = mpcalc.divergence(U, V)
######
# Vorticity
#vort = mpcalc.vorticity(U, V)
######
# Potential Temperature 
#pottemp = mpcalc.potential_temperature(plevel*units.hPa,T)
######
# Frontogenesis
#fgen = mpcalc.frontogenesis(pottemp,U,V)
#fgen = mpcalc.smooth_gaussian(fgen,5)
######
# Dewpoint
#td = mpcalc.dewpoint_from_specific_humidity(plevel*units.hPa,Q.metpy.convert_units('g/kg'))
#td = mpcalc.smooth_gaussian(td,20)
######
# RH
#RH = mpcalc.relative_humidity_from_specific_humidity(plevel*units.hPa,T,Q)
######
# Temperature Advection
tadv = mpcalc.advection(T, Ums, Vms)

In [ ]:
kelvin_to_fahrenheit = 9 / 5  # Conversion factor for K to °F
seconds_to_hours = 3600       # Conversion factor for seconds to hours

In [ ]:
temperature_adv_f_per_s = tadv * kelvin_to_fahrenheit

# Convert from °F/s to °F/hr
temperature_adv_f_per_hr = temperature_adv_f_per_s * seconds_to_hours
tadv=temperature_adv_f_per_hr

In [ ]:
tadv

In [ ]:
tadv.min(),tadv.max()

## **Graphing**

In [ ]:
contour_int = 1
contourf_int = .5
negtadvContours = np.arange (-12, -1, contourf_int)
postadvContours = np.arange (3, 29, contour_int)
advContours = np.arange (-12,28,contourf_int)

In [ ]:
#def segment_cmap(cmap_name, start, end):
    #cmap = plt.get_cmap(cmap_name)
    #colors = cmap(np.linspace(start, end))
    #return LinearSegmentedColormap.from_list(f"{cmap_name}_segment_{start}_{end}", colors)



In [ ]:
#segment_cmap = segment_cmap('RdPu', 0.4, 1)

In [ ]:
cmap1='Blues_r'
cmap2='Reds'
bounds = np.arange(0, 60, 5)
ticks = bounds


In [ ]:
frnt=tadv.fillna(0)

In [ ]:
for time in dateList:
    print("Processing", time)
    
    # Create the time strings, for the figure title as well as for the file name.
    timeStr = dt.strftime(time,format="%Y-%m-%d %H%M UTC")
    timeStrFile = dt.strftime(time,format="%Y%m%d%H")
    levstr=f'{plevel}'
    
    tl1 = str('CFSR, Valid at: '+ timeStr)
    tl2 = ("Temperature Advection °F/hr " + levstr + "mb")
    
    title_line = (tl1 + '\n' + tl2 + '\n')
    
    fig = plt.figure(figsize=(22,14)) # Increase size to adjust for the constrained lats/lons
    ax = plt.subplot(1,1,1,projection=proj_map)
    ax.set_extent ([lonW+constrainLon,lonE-constrainLon,latS+constrainLat,latN-constrainLat])
    ax.add_feature(cfeature.COASTLINE.with_scale(res))
    ax.add_feature(cfeature.STATES.with_scale(res),edgecolor='brown')
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature (cfeature.RIVERS.with_scale(res))
    ax.add_feature (cfeature.LAND.with_scale(res))
    ax.add_feature (cfeature.COASTLINE.with_scale(res))
    ax.add_feature (cfeature.LAKES.with_scale(res))
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature(cfeature.OCEAN)
    
 
   
    #cPosTAdv = ax.contour(lons, lats, tadv.sel(time=time), levels=postadvContours, colors='red', linewidths=3, transform=proj_data)
    #ax.clabel(cPosTAdv, inline=1, fontsize=12, fmt='%.0f')
    
    
    #cNegTAdv = ax.contour(lons, lats, tadv.sel(time=time), levels=negtadvContours, colors='blue', linewidths=3, transform=proj_data)
    #ax.clabel(cNegTAdv, inline=1, fontsize=12, fmt='%.0f')
    
    contourf1=ax.contourf(lons,lats,tadv.sel(time=time),levels=postadvContours,cmap=cmap2,transform=proj_data,extend='max')
    contourf2=ax.contourf(lons,lats,tadv.sel(time=time),levels=negtadvContours,cmap=cmap1,transform=proj_data,extend='min')
    
    cbar1 = plt.colorbar(contourf1, ax=ax)
    cbar.set_label('Positive Temperature Advection °F/hr ', rotation=270, labelpad=15)
    cbar2 = plt.colorbar(contourf2, ax=ax)
    cbar.set_label('Negative Temperature Advection °F/hr ', rotation=270, labelpad=15)
    
    # Wind barbs
    #skip = 4
    #ax.barbs(lons[::skip],lats[::skip],UKts.sel(time=time)[::skip,::skip].values, VKts.sel(time=time)[::skip,::skip].values, color='black',zorder=2,transform=proj_data)

    title = plt.title(title_line,fontsize=16)